# Customer churn analysis with low-rank tensor block hazard model

The repository is the official implementation of 
[Causal Customer Churn Analysis with Low-rank Tensor Block Hazard Model](https://arxiv.org/pdf/2405.11377v1), ICML, 2024.

Here we present an example for implementing the proposed framework and compared it with two common models (generalized linear model and Cox proportional hazard). A full list of comparison can be found in `sim.py`. 

---------------------------------

# Example

In [9]:
from functools import partial
import numpy as np
import pandas as pd
from scipy.special import expit
from utils import theta2prob, gen_data_potential_Y_binary, \
    TensorCompletionCovariateBinary, \
    get_theta_binary, get_theta_survival, \
        regime_eval, regime_prec
import scipy
from sim import main

## generate data N = 500, T = 10, k =3

In [2]:
N = 500; T = 10; k = 3; d = 3; seed = 2
Y, A, delta, X, theta_true, error = gen_data_potential_Y_binary(seed = seed, N = N, T = T, k = k, d = d)

## estimate the weight propensities

In [3]:
def weight_contd_grad(coef, XX_t, A_t):
        
        N = XX_t.shape[0]
        X_cov = np.hstack((np.ones((N, 1)), XX_t))
        # compute the weight
        linear_pred = np.dot(X_cov, coef)
        # obj_grad_CAL = (A_t/expit(linear_pred) -1).dot(XX_t)/N
        obj_grad_CAL = (A_t / expit(linear_pred) -1).dot(X_cov)/N
        # obj_grad_CAL = (expit(linear_pred) - A_t).dot(XX_t)/N
        return obj_grad_CAL
def rho_trt(kn):
    weights_coef1 = scipy.optimize.root(fun = partial(weight_contd_grad,
                                      XX_t = X, A_t = A[:, kn]),
                        x0 = np.zeros(d+1),
                        # jac = partial(weight_contd_hess,
                        #               XX_t = CV_X,
                        #               A_t = CV_A),
                        method = 'linearmixing').x
    weights_coef0 = scipy.optimize.root(fun = partial(weight_contd_grad,
                                      XX_t = X, A_t = 1 - A[:, kn]),
                        x0 = np.zeros(d+1),
                        # jac = partial(weight_contd_hess,
                        #               XX_t = CV_X,
                        #               A_t = CV_A),
                        method = 'linearmixing').x
    # compute the estimated propensity weights
    rho = expit(X @ weights_coef1[1:] + weights_coef1[0]) * A[:, kn] +\
        expit(X @ weights_coef0[1:] + weights_coef0[0]) * (1-A[:, kn]) 
    return rho
    
# focus on past k treatments
PS_trt = np.array([rho_trt(kn) for kn in range(k)]) 
rho = np.apply_along_axis(np.prod, 0, PS_trt)

## low-rank tensor block hazard model estimation

In [4]:
# replace Nan with zero for computational purpose
Y[np.isnan(Y)]= 0    
# weighted tensor block hazard model
TC_bry_w = TensorCompletionCovariateBinary(Y = Y, A = A, X = X, delta = delta,
                                           rho = rho,
                                           stepsize = 1e-5,     
                                           niters = 10000,
                                           r1_list = [4], r2_list = [2], r3_list = [k + 1])
theta_hat_w = TC_bry_w.SequentialTuning()

(CO-Tucker): 0th iteration with loss: 11.472
(CO-Tucker): 1000th iteration with loss: 9.598
(CO-Tucker): 2000th iteration with loss: 9.569
(CO-Tucker): 3000th iteration with loss: 9.551
(CO-Tucker): 4000th iteration with loss: 9.542
(CO-Tucker): 5000th iteration with loss: 9.539
(CO-Tucker): 6000th iteration with loss: 9.535
(CO-Tucker): 7000th iteration with loss: 9.532
(CO-Tucker): 8000th iteration with loss: 9.53
(CO-Tucker): 9000th iteration with loss: 9.528


In [30]:
# obtain the estimated survival curves of generalized linear model
est_prob_GLM = theta2prob(get_theta_binary(Y = Y, A = A, X = X, delta = delta,
                                          method = 'logit'))
# obtain the estimated survival curves of CoxPh model
est_prob_CoxPH = get_theta_survival(Y, A, X, delta, method = 'coxPH')
# obtain the estimated survival curves of our model
est_prob_TC = theta2prob(theta_hat_w)
# true survival curves
true_prob = theta2prob(theta_true)
# output the normalized tensor estimation error
print('Normalized tensor estimation error of GLM: {0:.4f}\n'.format(np.linalg.norm(est_prob_GLM - true_prob)/\
    np.linalg.norm(true_prob)))
print('Normalized tensor estimation error of CoxPH: {0:.4f}\n'.format(np.linalg.norm(est_prob_CoxPH - true_prob)/\
    np.linalg.norm(true_prob)))
print('Normalized tensor estimation error of ours: {0:.4f}\n'.format(np.linalg.norm(est_prob_TC - true_prob)/\
    np.linalg.norm(true_prob)))

Normalized tensor estimation error of GLM: 0.5791

Normalized tensor estimation error of CoxPH: 0.6063

Normalized tensor estimation error of ours: 0.2505

